In [ ]:
import numpy as np
import pandas as pd
import glob
import os
import re

import matplotlib.pyplot as plt
# plt.style.use('dark_background')  # heck yeah
%matplotlib widget

%load_ext autoreload
%autoreload 1

processed_directory = '/Users/cirorandazzo/code/callback-analysis/data/loom-only/*.wav'

channel_mic = 0
channel_loom_trig = 4

fs = 44100

files = [f for f in glob.glob(processed_directory)]
files.sort()

print(f'Found {len(files)} files!')

In [ ]:
%aimport utils.audio
%aimport utils.video

from utils.audio import AudioObject
from utils.video import get_video_frames_from_callback_audio
from scipy.signal import butter

#
savedir = './data/loom_figs'

# prep filter
f_low, f_high = (500, 15000)
b, a = butter(8, [f_low, f_high], btype="bandpass", fs=fs)

# smooth window
smw = round(2/1000 * fs)  # 2ms

# spectrogram params
n = 1024  # window length
overlap = 1020

In [ ]:
# Subplot with audio & loom trigger

# fig, axs = plt.subplots(nrows=2, ncols=1, sharex=True)

# axs[0].plot(audio.get_sample_times(), audio.audio_frs)
# axs[0].set(title="audio")

# axs[1].plot(loom_trig.get_sample_times(), loom_trig.audio)
# axs[1].scatter(loom_onsets / fs, loom_trig.audio[loom_onsets], c="red")

# axs[1].set(
#     title="loom trigger",
#     # xlim=(8.3, 9),
#     xlabel="Time since block onset (s)",
# )

In [ ]:
plt.close("all")

In [ ]:
f = files[1]
fig, ax = plt.subplots()

audio = AudioObject.from_wav(f, b=b, a=a, channel=channel_mic)
loom_trig = AudioObject.from_wav(f, channel=channel_loom_trig)

loom_onsets = get_video_frames_from_callback_audio(loom_trig.audio)

audio.rectify_smooth(smooth_window_f=smw)

ax.plot(audio.audio_frs)

In [ ]:
for f in files:
    audio = AudioObject.from_wav(f, b=b, a=a, channel=channel_mic)
    loom_trig = AudioObject.from_wav(f, channel=channel_loom_trig)

    loom_onsets = get_video_frames_from_callback_audio(loom_trig.audio)

    audio.rectify_smooth(smooth_window_f=smw)

    fig, ax = plt.subplots()

    ax.plot(audio.get_sample_times(), audio.audio_frs, label="Audio (FRS)")
    ax.scatter(
        loom_onsets / fs,
        audio.audio_frs[loom_onsets],
        c="red",
        label="Loom trigger onset",
    )

    xlim = np.array([np.min(loom_onsets), np.max(loom_onsets)])
    xlim = xlim / fs
    xlim[0] -= 1
    xlim[1] += 1

    ax.set(
        title=os.path.basename(f),
        xlim=xlim,
        ylim=[-1e7, 1e8],
        # ylim=[-1e8, 5e9],
        xlabel="Time since block onset (s)",
        ylabel="Audio amplitude",
    )
    ax.legend()

    fname = os.path.splitext(os.path.basename(f))[0] + ".png"
    fname = os.path.join(savedir, fname)

    # fig.savefig(fname)

In [ ]:
threshold = 5e5
post_loom_window_ms = 500

re_block = "((-|_|\/|\\)(Block)[0-9]{1,2})"

df = pd.DataFrame([], columns=["block", "loom_idx", "latency_ms", "magnitude", "file"])

for f in files:
    audio = AudioObject.from_wav(f, b=b, a=a, channel=channel_mic)
    loom_trig = AudioObject.from_wav(f, channel=channel_loom_trig)

    loom_onsets = get_video_frames_from_callback_audio(loom_trig.audio)

    audio.rectify_smooth(smooth_window_f=smw)

    block_search = re.search("(?:Block)([0-9]{1,2})", f)
    block = int(block_search.groups()[0])

    for i, loom in enumerate(loom_onsets):
        window_end = loom + round(post_loom_window_ms / 1000 * fs)

        # print(f"{loom} : {window_end} | {loom / fs} : {window_end / fs}")

        xlim = np.array([loom, window_end]) / fs
        fig, ax = plt.subplots()
        ax.plot(audio.get_sample_times(), audio.audio_frs)
        ax.set(xlim=xlim, xlabel="Time since block onset (s)", ylim=[0, 1e7])
        ax.plot(xlim, [threshold, threshold], "--k")

        sec_ax = ax.secondary_xaxis(
            "top",
            functions=(
                lambda x: (x - (loom / fs)) * 1000,
                lambda x: x / 1000 + (loom / fs),
            ),
        )
        sec_ax.set(xlabel="Time since loom trig (ms)")

        fig.savefig(os.path.join(savedir, "by_loom", f"block{block}-loom{i}.png"))
        plt.close()  # prevent display

        thresholded = np.flatnonzero(audio.audio_frs[loom:window_end] > threshold)

        # l = thresholded
        if len(thresholded) == 0:
            l = np.nan
        else:
            l = thresholded[0] * 1000 / fs

        m = np.max(audio.audio_frs[loom:window_end])

        df.loc[-1] = [block, i, l, m, f]
        df.index = df.index + 1
        df.sort_index(inplace=True)

df.set_index(keys=["block", "loom_idx"], inplace=True)
df.sort_index(inplace=True)
df

In [ ]:
import pickle
from datetime import datetime

now = datetime.now()
dt_str = now.strftime("%Y%m%d_%H%M%S")

pname = os.path.join(savedir, f"loom_only_data-{dt_str}.pickle")

with open(pname, "wb") as f:
    save = dict(
        df=df,
        threshold=5e5,
        post_loom_window_ms=500,
    )

    pickle.dump(save, f)

In [ ]:
# import pickle
# import pandas as pd

# pname = "./data/loom_figs/loom_only_data-20240815_120749.pickle"
# with open(pname, "rb") as f:
#     data = pickle.load(f)

# df = data["df"]
# df

In [ ]:
block_grid = True

fig, axs = plt.subplots(nrows=2, ncols=1, sharex=True)

df["plot_i"] = [3 * bl + li for (bl, li) in df.index]

axs[0].plot("plot_i", "latency_ms", data=df)
axs[0].set(ylabel="Latency to startle (ms)")

axs[1].plot("plot_i", "magnitude", data=df)
axs[1].set(xlabel="Loom Stimulus #", ylabel="FRS Audio Amplitude")

new_blocks = df.loc(axis=0)[:, 0]["plot_i"]  # get all loom_idx=0 regardless of block
sec_ax = axs[0].secondary_xaxis(location="top")
sec_ax.set(
    xlabel="Block",
    xticks=list(new_blocks),
    xticklabels=new_blocks.index.get_level_values("block"),
)

if block_grid:
    block_lines = dict(
        x=new_blocks,
        colors="k",
        linestyles="solid",
        alpha=0.2,
        linewidths=0.5,
        zorder=-1,
    )

    for ax in axs:
        ymin, ymax = ax.get_ylim()
        ax.vlines(ymin=ymin, ymax=ymax, **block_lines)
        ax.set_ylim(ymin, ymax)  # ensure grid stretches